In [ ]:
__author__ = 'Stephanie Juneau <stephanie.juneau@noirlab.edu>, David Herrera <david.herrera@noirlab.edu>, and the Astro Data Lab Team <datalab@noirlab.edu>'
__version__ = '20230424' # yyyymmdd
__datasets__ = ['ls_dr3','sdss_dr13']
__keywords__ = ['extragalactic','galaxies','joint query','spectroscopic redshift','3d plot']

# DECaLS and SDSS/BOSS Large Scale Structure
by Stéphanie Juneau, David Herrera, and the Astro Data Lab Team

### Table of contents
* [Goals & Summary](#goals)
* [Disclaimer & attribution](#attribution)
* [Imports & setup](#import)
* [Joint Query of LS and SDSS catalogs](#query)
* [Plot Results](#plots)
* [Exercise](#exercise)
* [3D plot (RA,DEC and z)](3d_plot)

<a class="anchor" id="goals"></a>
# Goals
* Joint query between photometric (LS) and spectroscopic (SDSS) catalogs
* Plot on-sky position of extragalactic objects, color-coded by redshift
* Plot, visualize and move around positions of extragalactic objects in 3D

# Summary

In this Notebook, we explore large-scale structures of galaxies by combining spectroscopic redshifts from SDSS/BOSS with photometry from the DESI pre-imaging Legacy Survey (LS). The advantage of spectroscopic redshifts is that they are far more accurate than photometric redshifts to probe distances to galaxies (though still need to be corrected for possible distortion effects such as the finger-of-God effect, which we ignore here). The advantage of the LS photometry is that it reaches deeper than SDSS by about 1 magnitude, which yields better image quality to measure magnitudes, colors, and galaxy shapes. While there are several possible extensions to the example work included below, we will show that a simple figure of galaxy spatial locations color-coded by galaxy morphological type reveals the known morphology-density relation.

We wanted to extend indeed a little further and be able to visualize and even interact with a representation of these galaxies in the actual space. For that, we developed a 3D plot based directly (as a flat cube, without any projection or correction) on RA, DEC and z, and that is at the end of this NB.

On a technical point of view, this short notebook illustrates an example joint query between the LS DR3 photometry Tractor table, and the SDSS/BOSS DR13 specObj spectroscopy table. It takes advantage of the fact that there is a version of the LS DR3 tractor table that was pre-matched to SDSS/BOSS DR13 so we can join on a common column called **specobjid**.

The columns from the LS table used (Tractor, pre-matched to specObj DR13) can be seen here: http://datalab.noirlab.edu/query.php?name=ls_dr3.dr3_specobj_dr13

The columns from the SDSS/BOSS DR13 table can be seen here: http://datalab.noirlab.edu/query.php?name=sdss_dr13.specobj

The columns from the SDSS DR17 used for the 3D plot can be found here http://datalab.noirlab.edu/query.php?name=sdss_dr17.specobj


<a class="anchor" id="attribution"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: http://datalab.noirlab.edu/disclaimers.php

<a class="anchor" id="import"></a>
# Imports and setup

Please note that this notebook is written for Python 3.

In [ ]:
# std lib
from getpass import getpass

# 3rd party
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.stats import binned_statistic_2d
%matplotlib inline
from astropy.table import Table
from astropy.cosmology import Planck18 as cosmo
import plotly
import plotly.graph_objs as go
import pandas as pd
plotly.offline.init_notebook_mode()

# Data Lab
from dl import queryClient as qc
from dl import authClient as ac

print('Done importing')

# Authentication

In [ ]:
# Uncomment the next 3 lines in case authentication is needed:
#token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
#if not ac.isValidToken(token):
#    raise Exception('Token is not valid. Please check your usename/password and execute this cell again.')

<a class="anchor" id="query"></a>
# Query DECaLS Tractor Photometry Catalog

The photometry is derived from Tractor modeling of sources, and the database includes model photometry, type (shape), as well as other quantities. 

The Legacy Survey DR3 database is called **ls_dr3** and includes several tables. We will use the **dr3_specobj_dr13** table, which is a version of the LS DR3 tractor table which was pre-matched with specObj table from SDSS/BOSS DR13. The column names and descriptions can be found from the [Data Lab Query Interface](http://datalab.noirlab.edu/query.php) or using the Table Access Protocol (TAP) service with, e.g., TOPCAT. In both cases, we are interested in **ls_dr3.dr3_specobj_dr13**. 

The SDSS DR13 database is called **sdss_dr13** and also includes several tables. We will use the **specObj** table, which has spectroscopic information.

In [ ]:
%%time
# number of rows from LS DR3 tractor (NOTE: tractor is the main photometry table):
query="SELECT nrows FROM tbl_stat WHERE schema='ls_dr4' and tbl_name='tractor'"

# Call query manager
response = qc.query(sql=query, fmt='csv')

print(response)
print('Time')

In [ ]:
%%time
# number of rows from SDSS specObj DR13:
query="SELECT nrows FROM tbl_stat WHERE schema='sdss_dr13' and tbl_name='specobj'"

# Call query manager
response = qc.query(sql=query, fmt='csv')

print(response)
print('Time')

In [ ]:
# ls_dr3.dr3_specobj_dr13           #DECaLS matched to SDSS DR13 specobj
# sdss_dr13.specobj                 #SDSS DR13 specobj

# Write query statement (adql)
query = ("""SELECT 
           L.ra,L.dec,L.type,L.g_r,L.r_z,
           S.z,S.plug_ra,S.plug_dec,S.class, 
           S.vdisp,S.vdisp_err 
         FROM ls_dr3.dr3_specobj_dr13 as L JOIN sdss_dr13.specobj as S 
         ON L.specObjId = S.specobjid 
         WHERE L.ra BETWEEN %s and %s and L.dec BETWEEN %s and %s and (L.ra_ivar > 0) 
         limit 100000""") %(126,131,7.,12.)  #small region

# L.ra, L.dec      = RA, Dec from Legacy Survey (LS) table    
# L.type           = object type (PSF, SIMP, EXP, DEV, COMP)
# L.g_r, L.r_z     = pre-computed g-r and r-z colors from photometry
# S.z              = redshift (z) from SDSS specObj table
# S.plug_ra,dec    = RA, Dec of SDSS fiber from specObj table
# S.class          = Source class (Star, Galaxy, QSO) from SDSS
# S.vdisp,vdisp_err= velocity dispersion (and error) from SDSS specObj table
#
# WHERE: requirement that RA & Dec coordinates are within a rectangular region

print(query)

In [ ]:
%%time
# Call query manager
response = qc.query(adql=query, fmt='csv')

print('Time')

In [ ]:
# Reformat output into a table
result = Table.read(response, format='csv')  #dictionary

result[:10]

<a class="anchor" id="plots"></a>
# Plot Results

## Sanity check: RA, Dec positions from both tables

In [ ]:
# convert RA coordinates from [0,360] to [-180,180] 
chgsign = np.where(result['ra'] > 180)
result['ra'][chgsign] = result['ra'][chgsign]-360.
result['plug_ra'][chgsign] = result['plug_ra'][chgsign]-360.

plt.figure(figsize=(9,8))

# plot RA, Dec from LS catalog in red with larger symbols
plt.scatter(result['ra'],result['dec'],s=3,color='red',marker='1')

# overplot RA, Dec from SDSS catalog in blue with smaller symbols
plt.scatter(result['plug_ra'],result['plug_dec'],s=3,color='black',marker='2')

# Extent of RA, Dec (in degrees) to plot
xmin = 126.
xmax = 131.
ymin = 7.
ymax = 12.

plt.axis([xmin, xmax, ymin, ymax])
plt.xlim(reversed(plt.xlim())) # flip the x-axis
plt.xlabel("RA (degrees)", fontsize=20)
plt.ylabel("Dec (degrees)", fontsize=20)
plt.show()

## Visual Inspection of Large-Scale Structures

Plot the positions of a broad range of redshift, and overplot a thin slice in redshift to show possible structures within that slice.

In [ ]:
# Select redshift slice
rz = np.where((result['z'] >0.105) & (result['z']<0.125))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6.5))

# plot all points in red (all redshifts)
ax1.scatter(result['plug_ra'],result['plug_dec'],s=3,color='r',marker='o',alpha=0.25)

# overplot in blue objects in narrow redshift slice
ax1.scatter(result['plug_ra'][rz],result['plug_dec'][rz],s=10,color='black')

# Extent of RA, Dec (in degrees) to plot
xmin = 126.
xmax = 131.
ymin = 7.
ymax = 12.

ax1.axis([xmin, xmax, ymin, ymax])
ax1.set_xlim(reversed(ax1.set_xlim())) # flip the x-axis
ax1.set_xlabel("RA (degrees)", fontsize=20)
ax1.set_ylabel("Dec (degrees)", fontsize=20)

# add rectangle to show where we zoom in next panel
ax1.add_patch(patches.Rectangle((128.65-0.25, 8.85-0.2),0.5,0.4,fill=False,color='b'))


## ZOOM IN A SMALLER REGION

# plot all points in red (all redshifts)
ax2.scatter(result['plug_ra'],result['plug_dec'],s=15,color='r',marker='o',alpha=0.3)

# overplot in blue objects in narrow redshift slice
ax2.scatter(result['plug_ra'][rz],result['plug_dec'][rz],s=30,color='black')

# Extent of RA, Dec (in degrees) to plot
xmin = 128.4
xmax = 128.9
ymin = 8.65
ymax = 9.05

ax2.axis([xmin, xmax, ymin, ymax])
ax2.set_xlim(reversed(ax2.set_xlim())) # flip the x-axis
ax2.set_xlabel("RA (degrees)", fontsize=20)
ax2.set_ylabel("Dec (degrees)", fontsize=20)

# add rectangle to show where we zoom in next panel
ax2.add_patch(patches.Rectangle((128.63, 8.81),0.13,0.107,fill=False,color='b'))

plt.show()

Above, the left-hand panel shows a thin redshift slice (0.105 < *z* < 0.125, black symbols) among objects with redshifts from the spectroscopic SDSS DR13 sample (red symbols). We can see by eye some large-scale filamentary structures and overdensities. The blue rectangle shows a selected region where we zoom in the right-hand panel. On the latter, we further select a smaller region, which we will use in the next cell below.

In [ ]:
## ZOOM IN AGAIN OVER AN EVEN SMALLER REGION
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5.5))

# plot all points in red (all redshifts)
ax1.scatter(result['plug_ra'],result['plug_dec'],s=30,color='r',marker='o',alpha=0.25)

# overplot in black objects in narrow redshift slice
ax1.scatter(result['plug_ra'][rz],result['plug_dec'][rz],s=50,color='black')

# Extent of RA, Dec (in degrees) to plot
xmin = 128.63
xmax = 128.76
ymin = 8.81
ymax = 8.917

ax1.axis([xmin, xmax, ymin, ymax])
ax1.set_xlim(reversed(ax1.set_xlim())) # flip the x-axis
ax1.set_xlabel("RA (degrees)", fontsize=20)
ax1.set_ylabel("Dec (degrees)", fontsize=20)

## SHOW DECaLS IMAGE (screenshot pre-made but could instead implement image cutout)
im = plt.imread('DECaLS_screenshot_zoomIn_labels.jpg')
ax2.imshow(im)
ax2.axis('off')

plt.show()

The left-hand panel shows the small region enclosed in the blue rectangle that we chose above (right-hand panel). The galaxies in black are in the same narrow redshift slice as defined previously (0.105 < *z* < 0.125). The right-hand panel is an image cutout of the same region of the sky from the [LS sky viewer](http://legacysurvey.org/viewer?ra=128.7040&dec=8.8439&zoom=13&layer=decals-dr5). The galaxies encircled correspond to the points in black, and some or perhaps most of them likely belong to a galaxy cluster.

## Large-Scale Structures with LS Morphologies

There are many possible extensions to this work. For instance, one could plot again with symbols coded with object *type* (from LS) and/or *class* (from SDSS) and/or velocity dispersion (from SDSS) and/or other quantities. Here, we will start with the object "TYPE" from LS, related to the morphological shapes. 

The object shape (2D light profile) is modeled by [the Tractor](http://thetractor.org) ([Lang, Hogg & Mykytyn](http://adsabs.harvard.edu/abs/2016ascl.soft04008L)) as part of the procedure to compute model photometry.

Possible shapes for LS DR3:
<ul>
<li>PSF (point spread function: size will vary with the seeing of the observations)</li>
<li>SIMP (“simple” galaxies: round, exponential profile with 0.45″ effective radius)</li>
<li>EXP (exponential profile; spiral galaxies)</li>
<li>DEV (deVaucouleurs profile; elliptical galaxies)</li>
<li>COMP (composite deVaucouleurs+exponential at same centroid)</li>
</ul>

Please note that starting in DR4 and in subsequent data releases, the SIMP model as been replaced with a REX model (Round Exponential).

<figure>
<img src="Gals_Qso.jpg", align="left", width=600>
<figcaption>**Figure:** Images of galaxies including a nearby elliptical galaxy, a nearby spiral galaxy, and a QSO.</figcaption>
</figure>

In [ ]:
# Select redshift slice
rz = np.where((result['z'] >0.105) & (result['z']<0.125))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6.5))

# plot all points in red (all redshifts)
ax1.scatter(result['plug_ra'],result['plug_dec'],s=3,color='r',marker='o',alpha=0.25)

# overplot in black objects in narrow redshift slice
ax1.scatter(result['plug_ra'][rz],result['plug_dec'][rz],s=10,color='black')

# Extent of RA, Dec (in degrees) to plot
xmin = 126.
xmax = 131.
ymin = 7.
ymax = 12.

ax1.axis([xmin, xmax, ymin, ymax])
ax1.set_xlim(reversed(ax1.set_xlim())) # flip the x-axis
ax1.set_xlabel("RA (degrees)", fontsize=20)
ax1.set_ylabel("Dec (degrees)", fontsize=20)


# Select redshift slice and per type
rdev  = np.where((result['type']=='DEV') & (result['z'] >0.105) & (result['z']<0.125))
rexp  = np.where((result['type']=='EXP') & (result['z'] >0.105) & (result['z']<0.125))
rcomp = np.where((result['type']=='COMP') & (result['z'] >0.105) & (result['z']<0.125))

# plot all points in red (all redshifts)
ax2.scatter(result['plug_ra'],result['plug_dec'],s=3,color='gray',marker='o',alpha=0.25)

# overplot in blue objects in narrow redshift slice
ax2.scatter(result['plug_ra'][rexp],result['plug_dec'][rexp],s=15,color='b')         # blue = EXP
ax2.scatter(result['plug_ra'][rdev],result['plug_dec'][rdev],s=15,color='r')         # red  = DEV
ax2.scatter(result['plug_ra'][rcomp],result['plug_dec'][rcomp],s=15,color='orange')  # orange = COMP

plt.axis([xmin, xmax, ymin, ymax])
plt.xlim(reversed(plt.xlim())) # flip the x-axis
plt.xlabel("RA (degrees)", fontsize=20)
plt.ylabel("Dec (degrees)", fontsize=20)
plt.show()

<a class="anchor" id="exercise"></a>
## EXERCISE: Large-Scale Structures with Galaxy Colors

There are pre-computed colors available. The columns are described here:
http://datalab.noirlab.edu/query.php?name=ls_dr3.dr3_specobj_dr13

Another possibility would be to plot again the galaxies spatial coordinates, but color-coded according to their photometric colors. This is left as an exercise for the user, but feel free to get in touch with the Data Lab Team if you have questions.

<a class="anchor" id="3d_plot"></a>
# Interactive 3D plot (RA, DEC and z)

We now turn to a different area of the sky. We want to identify filaments and clumps of galaxies in 3D, moving around and 'traveling' through space.

We want to take an interesting section of the sky, where we could observe galaxies clustering and/or filaments of the large scale structure of the universe, as we did in the previous section, except that now we can move through 'space' within those galaxies and see them from different points of view in that section of the universe.

We chose an are far from the galactic plane of the Milky Way

Next, we will get the SDSS data for positions and redshifts, and define our boundaries.

### Plotting in 3D
We want to take an interesting section of the sky, where we could observe galaxies clustering and/or filaments of the large scale structure of the universe, as we did in the previous section, EXCEPT, that now we can move through 'space' within those galaxies and see them from different points of view in that section of the universe.

In the following cell, we will get the SDSS data for coordinates and redshift, and define our boundaries:

In [ ]:
#Create the query to fetch the SDSS data from DataLab:

query = """
SELECT ra,dec,z
FROM sdss_dr17.specobj
WHERE q3c_radial_query(ra,dec,%s,%s,%s)
""" % (190,8,8)
print (query)

In [ ]:
%%time
# Fetch the SDSS data from ls_dr17.specobj
selection = qc.query(adql=query, fmt='csv')

In [ ]:
# Reformat output into a table
data = Table.read(selection, format='csv')  #dictionary
data[:10]

In [ ]:
ra = data['ra']
dec = data['dec']
z = data['z']

ra_ = ra
dec_ = dec
z_ = z
print('There initially are %s points to plot'%len(ra_))

sel = (z_ > 0.02) & (z_ < 0.4)
ra_ = ra_[sel]
dec_ = dec_[sel]
z_ = z_[sel]

print('Now here are %s points to plot'%len(ra_))

Compute luminosity distance for every galaxy, using Planck cosmology values

In [ ]:
dist = cosmo.luminosity_distance(z_)
dist

In [ ]:
ra = ra_
dec = dec_

In [ ]:
print('There are %s points to plot'%len(ra))

In [ ]:
dist.min()

Convert ra,dec,dist to cartesian coordinates X,Y,Z (in Mpc)

In [ ]:
def get_cartesian(ra,dec,dist):
    rarad = np.radians(ra)
    decrad = np.radians(dec)
    X = dist * np.sin(decrad) * np.cos(rarad)
    Y = dist * np.sin(decrad) * np.sin(rarad)
    Z = dist * np.cos(decrad)
    return X,Y,Z

In [ ]:
X,Y,Z = get_cartesian(ra,dec,dist)

### Plotting in 3D
We want to take an interesting section of the sky, where we could observe galaxies clustering and/or filaments of the large scale structure of the universe, as we did in the previous section, EXCEPT, that now we can move through 'space' within those galaxies and see them from different points of view in that section of the universe.

In the following cell, we will get the SDSS data for coordinates and redshift, and define our boundaries:

Now we prepare the trace to plot

In [ ]:
trace = go.Scatter3d(
    x = X,
    y = Y,
    z = Z,
    mode = 'markers',
    marker = {
        'size'      : 1.5,
        'opacity'   : 0.7,
        'color'     : dist, 
        'colorscale': 'OrRd'
    }
)

In [ ]:
data = [trace]

Next, we define camera location and the layout of the plot

In [ ]:
camera = dict(
    up = dict(x = 0, y = 0, z = 1),
    center = dict(x = 0, y = 0, z = 0),
    eye = dict(x = -0.2, y = -0.2, z = -1)
)

layout = go.Layout(
    scene = dict(
        xaxis = dict(title = 'X',
                     backgroundcolor = 'black',
                     showbackground=True,
                     gridcolor = "black",
                     ),
        yaxis = dict(title = 'Y',
                     backgroundcolor = 'black',
                     showbackground=True,
                     gridcolor = "black",
                     ),
        zaxis = dict(title = 'Z',
                     backgroundcolor = 'black',
                     showbackground=True,
                     gridcolor = "black",
                     ),
    ),
        scene_camera = camera,
        plot_bgcolor = 'black',
        paper_bgcolor = 'black',
        title = None,
        showlegend=False,
        width = 1000,
        height = 1000,
        autosize = False,
        margin = {'l':0, 'r':0, 'b':0, 't':0}
)

In [ ]:
# Now we finally draw the plot:

plot_figure = go.Figure(data = data, layout = layout)
plot_figure.update_layout()


Feel free to zoom in and zoom out with your mouse/pad.  
You can immerse yourself in the plot!!  
Rotate and fly through the points, so you can notice the filaments and clusters from different points of view.  
Hover over different points and a pop up tag will reveal its RA, Dec and z

NOTE: Try to see the z axis from the side and notice there is a 'layer' effect in z. It is NOT clear why that is